# Natural Language Processing (NLP): Sentiment Analysis

## Project Overview

In this project, we’ll perform **Sentiment Analysis** — a foundational task in NLP that determines whether a piece of text expresses a **positive**, **negative**, or **neutral** opinion.  
We’ll use three approaches to show the evolution of NLP models:

1. **Naive Bayes** — a simple statistical model that assumes word independence.  
2. **Logistic Regression** — a linear classifier that uses word frequencies or TF-IDF features.  
3. **Transformer (BERT)** — a pre-trained deep learning model that understands context at the sentence level.

The goal is to classify text reviews and compare how traditional machine learning stacks up against modern transformers built with **PyTorch**.

---

## Dataset Description

**Dataset Name:** IMDb Movie Reviews Dataset  
**Source:** [Kaggle IMDb Dataset of 50K Movie Reviews](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)  
**Size:** 50,000 labeled reviews (balanced 25K positive, 25K negative)  
**Target Variable:** `sentiment` (values: *positive* or *negative*)

Each review contains raw English text written by users.  
We’ll preprocess the data (clean text, tokenize, and remove stopwords), vectorize it using TF-IDF, and then train and evaluate the models.

---

## Objective and Predictions

The objective is to **predict sentiment polarity** based on review text.  
We’ll compare:
- Accuracy, precision, recall, and F1-score of each model.  
- Model interpretability (which words drive predictions).  
- Training speed and computational efficiency.

---

## Hypothesized Conclusions

1. **Naive Bayes** will perform surprisingly well for its simplicity, reaching ~85–88% accuracy.  
2. **Logistic Regression** will outperform Naive Bayes slightly, as it better models correlated word features.  
3. **BERT (PyTorch)** will achieve the best accuracy (~93–96%) by understanding context, negations, and tone.

---

## Why We Use These Models for This Dataset

| Model | Why It’s Used |
|--------|----------------|
| **Naive Bayes** | Simple baseline for word-based classification. Great for bag-of-words features. |
| **Logistic Regression** | Improves on Naive Bayes by weighting words more flexibly. |
| **BERT (Transformer)** | Understands full sentence meaning and context; ideal for nuanced human language. |

In simple terms:
- Naive Bayes counts word probabilities.
- Logistic Regression balances words mathematically.
- BERT *understands* what you mean.

---

This notebook will go through:
1. Data loading and exploration  
2. Text preprocessing and tokenization  
3. Model training (Naive Bayes → Logistic Regression → BERT with PyTorch)  
4. Evaluation and comparison of results  

When you’re ready, we’ll begin by **loading and inspecting the dataset** from your `data/` directory.


---
---

## Data Loading and Initial Exploration

We'll begin by loading the **IMDb Movie Reviews** dataset from the local `data/` directory.  
This dataset contains 50,000 movie reviews labeled as positive or negative.  
Before we jump into preprocessing, we’ll inspect a few samples, check label distribution, and ensure that text lengths are balanced enough for modeling.


In [1]:
import pandas as pd

# Load IMDb dataset
df = pd.read_csv("data/IMDB Dataset.csv")

# Basic overview
print("Dataset Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nMissing Values:", df.isnull().sum().sum())

# Check sentiment distribution
print("\nSentiment Distribution:")
print(df['sentiment'].value_counts(normalize=True) * 100)

# Display a few examples
display(df.head())


Dataset Shape: (50000, 2)

Columns: ['review', 'sentiment']

Missing Values: 0

Sentiment Distribution:
sentiment
positive    50.0
negative    50.0
Name: proportion, dtype: float64


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


### Interpretation

- The dataset contains **50,000 reviews** evenly split between positive and negative sentiment — perfect for binary classification.  
- Each entry includes raw, unprocessed English text. Some reviews may contain punctuation, HTML tags, or mixed casing.  
- There are **no missing values**, so we can move directly to cleaning and preparing the text.

Next, we’ll preprocess the data — cleaning, tokenizing, and vectorizing the text so models can learn from it effectively.


---
---

## Text Preprocessing and Vectorization

Machine learning models can’t directly understand raw text — we need to **convert words into numbers**.  
We’ll clean the reviews by:
1. Lowercasing text  
2. Removing punctuation, HTML tags, and special symbols  
3. Tokenizing words (splitting text into individual words)  
4. Removing common stopwords (like “the”, “and”, “is”)  
5. Converting words into TF-IDF vectors — a numerical format representing how important each word is across all reviews.


In [2]:
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk

# Download stopwords if needed
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Text cleaning function
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # remove HTML tags
    text = text.lower()                # lowercase
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)  # remove punctuation
    text = re.sub(r'\d+', '', text)    # remove digits
    text = ' '.join([word for word in text.split() if word not in stop_words])  # remove stopwords
    return text

# Apply cleaning
df['cleaned_review'] = df['review'].apply(clean_text)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_review'], df['sentiment'], 
                                                    test_size=0.2, random_state=42, stratify=df['sentiment'])

# TF-IDF vectorization
tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1,2))
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print("TF-IDF feature matrix shape:", X_train_tfidf.shape)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gardi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TF-IDF feature matrix shape: (40000, 10000)


### Interpretation

Now every review is represented as a **TF-IDF feature vector** — essentially a long list of word importance scores.  
The model can now recognize which words are strong indicators of positive or negative sentiment (like *“amazing”*, *“awful”*, *“boring”*, etc.).

In short, we’ve turned messy English text into structured numerical data the models can actually learn from.

Next, we’ll train a **Naive Bayes classifier** — a fast, classic baseline model for text classification.


---
---

## Naive Bayes Model — Statistical Baseline for Text Classification

We’ll start with **Multinomial Naive Bayes**, one of the most popular and effective algorithms for text classification.  
It works by using **word frequencies** to calculate the probability that a review belongs to a certain class (positive or negative).  

Even though it assumes all words are independent (which isn’t true in real language), Naive Bayes often performs surprisingly well on large text datasets.


In [4]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize and train Naive Bayes
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

# Predictions
y_pred_nb = nb.predict(X_test_tfidf)

# Evaluation
acc_nb = accuracy_score(y_test, y_pred_nb)
print(f"Naive Bayes Accuracy: {acc_nb:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_nb))

# Confusion Matrix
cm_nb = confusion_matrix(y_test, y_pred_nb)
print("Confusion Matrix:\n", cm_nb)


Naive Bayes Accuracy: 0.8655

Classification Report:
              precision    recall  f1-score   support

    negative       0.88      0.85      0.86      5000
    positive       0.86      0.88      0.87      5000

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000

Confusion Matrix:
 [[4254  746]
 [ 599 4401]]


### Interpretation

Typical performance for Naive Bayes on IMDb reviews is around **85–88% accuracy**.  
The model learns to associate positive words (like *“great”*, *“excellent”*, *“love”*) with positive sentiment and negative words (like *“bad”*, *“boring”*, *“terrible”*) with negative sentiment.

In simple terms:  
Naive Bayes reads a review like a word counter — it tallies positive and negative words and chooses whichever side wins.  

While fast and interpretable, it doesn’t understand word order or context (e.g., *“not good”* still looks positive to it).

Next, we’ll improve on this with **Logistic Regression**, which gives each word a learned weight instead of assuming equal importance.


---
---

## Logistic Regression — Weighted Word Importance Model

While Naive Bayes treats all words independently, **Logistic Regression** learns a **weight** for each word or phrase (n-gram) to better capture subtle relationships.  
For example, it can tell that “not good” means something different from “good” — which Naive Bayes can’t do.

We’ll use the same TF-IDF features but train a linear model that directly optimizes classification accuracy.


In [5]:
from sklearn.linear_model import LogisticRegression

# Initialize and train Logistic Regression
log_reg = LogisticRegression(max_iter=200, n_jobs=-1)
log_reg.fit(X_train_tfidf, y_train)

# Predictions
y_pred_lr = log_reg.predict(X_test_tfidf)

# Evaluation
acc_lr = accuracy_score(y_test, y_pred_lr)
print(f"Logistic Regression Accuracy: {acc_lr:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))

# Confusion Matrix
cm_lr = confusion_matrix(y_test, y_pred_lr)
print("Confusion Matrix:\n", cm_lr)


Logistic Regression Accuracy: 0.8958

Classification Report:
              precision    recall  f1-score   support

    negative       0.90      0.89      0.89      5000
    positive       0.89      0.91      0.90      5000

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000

Confusion Matrix:
 [[4427  573]
 [ 469 4531]]


### Interpretation

Logistic Regression typically achieves **88–90% accuracy**, slightly outperforming Naive Bayes.  
It gives **each word a unique learned weight**, allowing it to distinguish between nuanced expressions such as “not great” vs. “great.”

In simpler terms:  
Naive Bayes votes based on word counts,  
while Logistic Regression *learns how strongly each word pushes the review toward positive or negative.*

This model is still lightweight and interpretable, making it a strong choice for production systems needing transparency.

Next, we’ll step up to **BERT (Transformer)** using **PyTorch**, a modern deep learning model that understands language context, meaning, and tone at a much deeper level.


---
---

## Transformer Model (BERT) — Contextual Deep Learning for Sentiment Analysis

Now we’ll use **BERT (Bidirectional Encoder Representations from Transformers)** — a state-of-the-art NLP model from Google, built on the Transformer architecture.  
Unlike traditional models, BERT reads text **in both directions** (left-to-right and right-to-left), understanding *context* rather than just word frequency.

We’ll fine-tune a pre-trained BERT model using **PyTorch** and **Hugging Face Transformers**, leveraging modern optimizers like **AdamW replaced with torch.optim.Adam** (the current best practice).


In [8]:
!pip install torch torchvision torchaudio transformers --quiet


In [9]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import Adam
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Sample subset for training (to fit in notebook memory)
df_sample = df.sample(10000, random_state=42).reset_index(drop=True)

# Tokenizer and encoding
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encodings = tokenizer(list(df_sample['cleaned_review']), truncation=True, padding=True, max_length=128)

# Convert sentiments to binary labels
labels = torch.tensor(df_sample['sentiment'].map({'negative': 0, 'positive': 1}).values)


Using device: cpu


In [10]:
# Create a custom PyTorch Dataset
class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

dataset = IMDbDataset(encodings, labels)

# Train/test split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [11]:
# Initialize BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)

# Optimizer (AdamW deprecated → use Adam)
optimizer = Adam(model.parameters(), lr=2e-5)

# Training loop
epochs = 2
model.train()
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2


  0%|          | 0/1000 [00:00<?, ?it/s]C:\Users\Gardi\AppData\Local\Temp\ipykernel_15800\1113597505.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
Epoch 1: 100%|██████████| 1000/1000 [38:55<00:00,  2.34s/it, loss=0.518]


Epoch 2/2


Epoch 2: 100%|██████████| 1000/1000 [39:38<00:00,  2.38s/it, loss=0.109] 


In [12]:
# Evaluation
model.eval()
preds, truths = [], []
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)
        preds.extend(predictions.cpu().numpy())
        truths.extend(labels.cpu().numpy())

# Metrics
acc_bert = accuracy_score(truths, preds)
print(f"BERT Accuracy: {acc_bert:.4f}")
print("\nClassification Report:")
print(classification_report(truths, preds))


  0%|          | 0/250 [00:00<?, ?it/s]C:\Users\Gardi\AppData\Local\Temp\ipykernel_15800\1113597505.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
100%|██████████| 250/250 [02:20<00:00,  1.77it/s]

BERT Accuracy: 0.8540

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.79      0.84       985
           1       0.82      0.92      0.86      1015

    accuracy                           0.85      2000
   macro avg       0.86      0.85      0.85      2000
weighted avg       0.86      0.85      0.85      2000



### Interpretation

Typical BERT fine-tuning results:
- **Accuracy:** ~93–96% on IMDb reviews  
- **Balanced precision/recall**, meaning it handles both positive and negative sentiment equally well.  

BERT succeeds because it understands **context and tone** — it knows that *“This movie wasn’t bad at all”* is positive, not negative.

In simple terms:
- **Naive Bayes:** counts happy/sad words.  
- **Logistic Regression:** weighs those words more intelligently.  
- **BERT:** actually understands what the *sentence means.*

Next, we’ll compare all three models and summarize what each is best suited for, wrapping up with final conclusions.


---
---

## Comparison and Final Conclusions

Now that we’ve tested all three sentiment analysis models — **Naive Bayes**, **Logistic Regression**, and **BERT (PyTorch)** — let’s compare how they performed and what makes each one unique.  
Each model represents a milestone in NLP’s evolution, from simple word counting to deep contextual understanding.

---

### 📊 Model Performance Summary

| Model | Accuracy | Precision (avg) | Recall (avg) | F1-Score | Key Strength |
|--------|-----------|----------------|---------------|-----------|---------------|
| **Naive Bayes** | 0.8655 | 0.87 | 0.87 | 0.87 | Fast, simple, interpretable |
| **Logistic Regression** | 0.8958 | 0.90 | 0.90 | 0.90 | Weighted word modeling |
| **BERT (PyTorch)** | 0.8540 | 0.86 | 0.85 | 0.85 | Context-aware, deep semantics |

---

### 🎯 Observations

- **Naive Bayes** performed solidly for its simplicity — it’s still an excellent baseline for text classification.  
- **Logistic Regression** delivered the best results here, likely due to the strong TF-IDF features and limited BERT fine-tuning (only two epochs and 10K samples).  
- **BERT** underperformed slightly in this small-sample test because fine-tuning large language models requires **more data and training time** to fully adapt to the task.

---

### 🧠 Interpreting the Results (as if explaining to high school students)

- **Naive Bayes** is like counting positive and negative words — if a review says “amazing” a lot, it probably likes the movie.  
- **Logistic Regression** is smarter — it doesn’t just count; it *learns which words matter more.* It knows that “not bad” isn’t the same as “bad.”  
- **BERT** reads the review like a person. It doesn’t rely on counting words — it actually *understands context*. But just like a human, it needs more “reading practice” (training) to get really good.

---

### ⚙️ Why the Results Make Sense

| Factor | Impact |
|---------|--------|
| **Limited BERT fine-tuning (10K samples, 2 epochs)** | Not enough time to adapt; underfitting likely. |
| **TF-IDF strength for classical models** | Preprocessed word frequencies captured most sentiment clues. |
| **Dataset balance** | With 50/50 positive/negative reviews, simpler models already perform well. |
| **GPU/epoch constraints** | BERT benefits significantly from longer fine-tuning (3–5 epochs, full 50K samples). |

---

### 🔧 How to Improve BERT for Better Results

1. **Fine-tune on full dataset (50K samples)** for at least 3–4 epochs.  
2. **Use DistilBERT or RoBERTa** — smaller, faster models often match full BERT accuracy on sentiment tasks.  
3. **Add learning rate warmup and gradient clipping** for stability.  
4. **Use mixed precision training (AMP)** if running on GPU for faster convergence.  
5. **Include data augmentation** (e.g., synonym replacement, random word swap) to expose BERT to more phrasing diversity.

---

### 📘 Portfolio Summary — Sentiment Analysis (IMDb Reviews)

Built and compared three models for sentiment analysis using IMDb’s 50K labeled movie reviews:  
- **Naive Bayes (86.5%)** – simple probability-based baseline using word counts.  
- **Logistic Regression (89.6%)** – improved accuracy by weighting words intelligently via TF-IDF.  
- **BERT (85.4%)** – achieved contextual understanding but underperformed due to limited fine-tuning.  

This project demonstrates hands-on expertise across **classic NLP pipelines** and **modern transformer-based modeling** using **PyTorch** — from preprocessing raw text to evaluating real-world sentiment prediction performance.

---

Would you like to begin the next portfolio project — **Computer Vision (Image Classification)** using CNN, ResNet, and Vision Transformer (ViT)?  
We’ll use PyTorch throughout to stay consistent with your framework preference.
